# Setup Environment and Dependencies

1. Subir un archivo con nombre env que tenga la clave de OpenAI
2. Renombrarlo a .env
3. Subir usina.db para que esten los datos disponibles

In [ ]:
%pip install langchain langchain_community langchain_openai typing_extensions python-dotenv

: 

In [ ]:
# Import required libraries
from typing_extensions import Annotated, TypedDict
from langchain import hub
from langchain.chat_models import init_chat_model
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize Database and LLM
Initialize the LLM model and connect to the SQLite database containing organizational data.

In [ ]:
# Initialize the LLM model
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

# Connect to the SQLite database
db = SQLDatabase.from_uri("sqlite:///usina.db")

# Define Data Structures
Define the State and QueryOutput TypedDict classes for managing data flow through the application.

In [ ]:
# Define the State TypedDict for managing data flow through the application
class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

# Define the QueryOutput TypedDict for the generated SQL query
class QueryOutput(TypedDict):
    """Generated SQL query."""
    query: Annotated[str, ..., "Syntactically valid SQL query."]

# Implement Query Generation
Implement the write_query function to generate SQL queries from natural language questions using LangChain prompts.

In [ ]:
# Implement the write_query function to generate SQL queries from natural language questions using LangChain prompts
def write_query(state: State):
    """Generate SQL query to fetch information."""
    query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return result["query"]

# Execute SQL Queries
Implement the execute_query function that uses QuerySQLDatabaseTool to run SQL queries against the database.

In [ ]:
# Implement the execute_query function to run SQL queries against the database using QuerySQLDatabaseTool
def execute_query(state: State):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return execute_query_tool.invoke(state["query"])

# Generate Natural Language Answers
Implement the generate_answer function to transform SQL results into human-readable answers using the LLM.

In [ ]:
# Implement the generate_answer function to transform SQL results into human-readable answers using the LLM
def generate_answer(state: State):
    """Answer question using retrieved information as context."""
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f'Question: {state["question"]}\n'
        f'SQL Query: {state["query"]}\n'
        f'SQL Result: {state["result"]}'
    )
    response = llm.invoke(prompt)
    return response.content

# Interactive Application
Create an interactive interface to accept user questions and display answers with intermediate SQL queries and results.

In [ ]:
# Interactive Application

import ipywidgets as widgets
from IPython.display import display

# Create input widget for user question
question_input = widgets.Text(
    value='',
    placeholder='Type your question here',
    description='Question:',
    disabled=False
)

# Create output widgets for displaying intermediate SQL query, results, and final answer
query_output = widgets.Output()
result_output = widgets.Output()
answer_output = widgets.Output()

# Function to handle user input and display results
def on_submit(change):
    state = State()
    state["question"] = question_input.value
    state["query"] = write_query(state)
    
    with query_output:
        query_output.clear_output()
        print(f"Generated SQL Query: {state['query']}")
    
    state["result"] = execute_query(state)
    
    with result_output:
        result_output.clear_output()
        print(f"SQL Query Result: {state['result']}")
    
    state["answer"] = generate_answer(state)
    
    with answer_output:
        answer_output.clear_output()
        print(f"Answer: {state['answer']}")

# Link the function to the input widget
question_input.on_submit(on_submit)

# Display the widgets
display(question_input, query_output, result_output, answer_output)